# Midterm Project

## a) Connect to the Twitter API and load a list of positive and negative words.


In [ ]:
import tweepy
from time import sleep
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:

# creating the 4 keys
consumer_key = "INSERT HERE"
consumer_secret = "INSERT HERE"
access_token = "INSERT HERE"
access_token_secret = "INSERT HERE"

# authenticating
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
# splitlines() remove the breakline character \n from each list of words
file = open('positive-words.txt', 'r')
positive_words = file.read().splitlines() 

file = open('negative-words.txt', 'r')
negative_words = file.read().splitlines() 

## b)	Write some code that continuously collects 5 tweets associated with a hashtag.


In [ ]:
import time

response = api.rate_limit_status()

remaining_searches = response['resources']['search']['/search/tweets']

# printing the search-related constraints
print(remaining_searches)
print("Reset time: " + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(remaining_searches['reset'])))

In [ ]:
# starting the infinite loop
cumulative_score = 0
corpus = []
df = []
while True:
    # retrieve 5 tweets
    search_results = api.search_tweets(q="#ISA414", lang="en", count=5)

    # for each tweet in my search
    for tweet in search_results:
        # verify whether each individual tweet is new
        if tweet.id not in df:
            df.append(tweet.id)
            # create corpus of single tweet
            corpus = [tweet.text]
            vectorizer   = TfidfVectorizer()
            tfidf_values = vectorizer.fit_transform(corpus)
            # calculate the score value
            all_word_names  = vectorizer.get_feature_names_out()
            #print(all_word_names)
            doc0_word_index = tfidf_values[0,:].nonzero()[1]
            #print(doc0_word_index)
            doc0_word_names = [all_word_names[w] for w in doc0_word_index]
            #print(doc0_word_names)
            # counting the number of times each word in document 80 occurs in the positive and negative lists of words
            count_positive_words = [positive_words.count(word) for word in doc0_word_names]
            count_negative_words = [negative_words.count(word) for word in doc0_word_names]

            # taking the difference between the sum of the above counts
            score = sum(count_positive_words) - sum(count_negative_words)
            #print(score)

            # updating cumulative score
            cumulative_score = cumulative_score + score

            # print new tweets and their scores
            print("Tweet: ",corpus[0], "\t","Score: " ,score)



    # print current cumulative score
    print("Current cumulative score is: ", cumulative_score)

        



    sleep(10)

In [ ]:
#search_results